# Run interventions in loop with OpenABM 

In [1]:
import sys
sys.path.insert(0,'./src/')
sib_folder = "../sib" # sib path
abm_folder = "../OpenABM-Covid19/src" #Open ABM path
sys.path.insert(0,sib_folder)
sys.path.insert(0,abm_folder)
from pathlib import Path
import log, logging
from importlib import reload
import loop_abm, abm_utils, scipy

#logging
output_dir = "./output/"
fold_out = Path(output_dir)
if not fold_out.exists():
    fold_out.mkdir(parents=True)

reload(log)
logger = log.setup_logger()

num_threads = 40 #number of threads used for sib 

Log setup complete


## Epidemic model

#### Set the parameters of the OpenABM foward simulation

In [2]:
N=50000 #Number of individuals
T=100 #Total time of simulations
seed = 1 #seed of the random number generator
n_seed_infection = 10 #number of patient zero

params_model = {
    "rng_seed" : seed,
    "end_time" : T,
    "n_total"  : N,
    "days_of_interactions" : T,
    "n_seed_infection" : n_seed_infection,
}

## Set testing and quarantine rules

In [3]:
fraction_SM_obs = 0.5 #fraction of Symptomatic Mild tested positive
fraction_SS_obs = 1 #fraction of Symptomatic Severe tested positive
initial_steps = 12 #starting time of intervention
quarantine_HH = True #Households quarantine
test_HH = True #Tests the households when quarantined
adoption_fraction = 1 #app adoption (fraction)
num_test_random = 0 #number of random tests per day
num_test_algo = 200 #number of tests using by the ranker per day
fp_rate = 0.0 #test false-positive rate
fn_rate = 0.0 #test false-negative rate

## Choose the rankers algorithms class

In [4]:
from rankers import dotd_rank, tracing_rank, mean_field_rank, sib_rank
import sib


prob_seed = 1/N
prob_sus = 0.55
pseed = prob_seed / (2 - prob_seed)
psus = prob_sus * (1 - pseed)
pautoinf = 1/N


dotd = dotd_rank.DotdRanker()

tracing = tracing_rank.TracingRanker(
                 tau=5,
                 lamb=0.014
)

MF = mean_field_rank.MeanFieldRanker(
                tau = 5,
                delta = 10,
                mu = 1/30,
                lamb = 0.014
                )

sib.set_num_threads(num_threads)
sib_ranker = sib_rank.SibRanker(
                 params = sib.Params(
                                 # faster implementation
                                 #prob_i = sib.Cached(sib.Scaled(sib.PDF(sib.Gamma(k=5.76, mu=0.96)), scale=0.25), T+1),
                                 #prob_r = sib.Cached(sib.Gamma(k = 10, mu = 1/1.7452974337097158), T+1),
                                 prob_i = sib.PiecewiseLinear(sib.RealParams(list(0.25*abm_utils.gamma_pdf_array(T+1,6,2.5)))),
                                 prob_r = sib.PiecewiseLinear(sib.RealParams(list(scipy.stats.gamma.sf(range(T+1), 10., scale=1.7452974337097158)))),
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=7
)


ress = {}


In [5]:
rankers = {
    "RG" : dotd,
    "CT": tracing,
    "SMF" : MF,
    "BP": sib_ranker,
} 

### Set up figure

In [6]:
%matplotlib widget
import matplotlib.pyplot as plt
import plot_utils
import time

plots = plot_utils.plot_style(N, T)
save_path_fig = f"./output/plot_run_N_{N}_SM_{fraction_SM_obs}_test_{num_test_algo}_n_seed_infection_{n_seed_infection}_seed_{seed}_fp_{fp_rate}_fn_{fn_rate}.png"
fig, callback = plot_utils.plotgrid(rankers, plots, initial_steps, save_path=save_path_fig)
time.sleep(0.5)
fig.canvas

Canvas(header_visible=False, toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Bac…

In [7]:
import imp 
imp.reload(loop_abm)

for s in rankers:
    data = {"algo":s}
    loop_abm.loop_abm(
        params_model, 
        rankers[s], 
        seed=seed, 
        logger = logging.getLogger(f"iteration.{s}"), 
        data = data, 
        callback = callback,
        initial_steps = initial_steps, 
        num_test_random = num_test_random,
        num_test_algo = num_test_algo,
        fraction_SM_obs = fraction_SM_obs,
        fraction_SS_obs = fraction_SS_obs,
        quarantine_HH = quarantine_HH,
        test_HH = test_HH,
        adoption_fraction = adoption_fraction,
        fp_rate = fp_rate,
        fn_rate = fn_rate,
        name_file_res = s + f"_N_{N}_T_{T}_obs_{num_test_algo}_SM_obs_{fraction_SM_obs}_seed_{seed}"
    )
    ress[s] = data
    # saves a bit of memory: rankers[s] = {}

COVID19.model: INFO     Returning self.c_params into Model object, future updates to parameters not possible
COVID19.model: INFO     Started model creation
COVID19.model: INFO     Successfuly created model
COVID19.model: INFO     Started model creation
COVID19.model: INFO     Successfuly created model
iteration.RG: INFO     time:0
iteration.RG: INFO     number of unique contacts: 632210
iteration.RG: INFO     True  : (S,I,R): (49990.0, 10.0, 0.0)
iteration.RG: INFO     AUR_I : 0.608, prec(1% of 50001): 0.00, prec5%: 0.00
iteration.RG: INFO     SS: 0, SM: 0, results test algo (S,I,R): (0,0,0), infected test random: 0/0
iteration.RG: INFO     false+: 0 (+0), false-: 0 (+0)
iteration.RG: INFO     ...quarantining 0 guys -> got 0 infected, 10 free as birds (+10)
iteration.RG: INFO     time:1
iteration.RG: INFO     number of unique contacts: 632414
iteration.RG: INFO     True  : (S,I,R): (49988.0, 12.0, 0.0)
iteration.RG: INFO     AUR_I : 0.387, prec(1% of 50001): 0.00, prec5%: 0.00
iteratio

sib.iterate(damp=0):  2/20 2.368e-07/0.001    
sib.iterate(damp=0.5):  0/20 7.297e-10/0.001    


iteration.BP: INFO     winBP: (S,I,R): (49999.1, 0.9, 0.0), seeds: 0.9, ll: -29891.6
iteration.BP: INFO     True  : (S,I,R): (49990.0, 10.0, 0.0)
iteration.BP: INFO     AUR_I : 0.426, prec(1% of 50001): 0.00, prec5%: 0.00
iteration.BP: INFO     SS: 0, SM: 0, results test algo (S,I,R): (0,0,0), infected test random: 0/0
iteration.BP: INFO     false+: 0 (+0), false-: 0 (+0)
iteration.BP: INFO     ...quarantining 0 guys -> got 0 infected, 10 free as birds (+10)
iteration.BP: INFO     time:1
iteration.BP: INFO     number of unique contacts: 632414


sib.iterate(damp=0):  2/20 3.025e-06/0.001    
sib.iterate(damp=0.5):  0/20 9.782e-08/0.001    


iteration.BP: INFO     winBP: (S,I,R): (49998.3, 1.7, 0.0), seeds: 0.9, ll: -29891.8
iteration.BP: INFO     True  : (S,I,R): (49988.0, 12.0, 0.0)
iteration.BP: INFO     AUR_I : 0.353, prec(1% of 50001): 0.00, prec5%: 0.00
iteration.BP: INFO     SS: 1, SM: 1, results test algo (S,I,R): (0,0,0), infected test random: 0/0
iteration.BP: INFO     false+: 0 (+0), false-: 0 (+0)
iteration.BP: INFO     ...quarantining 7 guys -> got 3 infected, 9 free as birds (-1)
iteration.BP: INFO     time:2
iteration.BP: INFO     number of unique contacts: 632688


...

iteration.BP: INFO     True  : (S,I,R): (49289.0, 2.0, 709.0)
iteration.BP: INFO     AUR_I : nan, prec(1% of 48930): 0.00, prec5%: 0.00
iteration.BP: INFO     SS: 0, SM: 0, results test algo (S,I,R): (200,0,0), infected test random: 0/0
iteration.BP: INFO     false+: 0 (+0), false-: 0 (+0)
iteration.BP: INFO     ...quarantining 0 guys -> got 0 infected, 0 free as birds (+0)
iteration.BP: INFO     time:98
iteration.BP: INFO     number of unique contacts: 612872


faster end


iteration.BP: INFO     True  : (S,I,R): (49289.0, 2.0, 709.0)
iteration.BP: INFO     AUR_I : nan, prec(1% of 48930): 0.00, prec5%: 0.00
iteration.BP: INFO     SS: 0, SM: 0, results test algo (S,I,R): (200,0,0), infected test random: 0/0
iteration.BP: INFO     false+: 0 (+0), false-: 0 (+0)
iteration.BP: INFO     ...quarantining 0 guys -> got 0 infected, 0 free as birds (+0)
iteration.BP: INFO     time:99
iteration.BP: INFO     number of unique contacts: 613076


faster end


iteration.BP: INFO     True  : (S,I,R): (49289.0, 1.0, 710.0)
iteration.BP: INFO     AUR_I : nan, prec(1% of 48930): 0.00, prec5%: 0.00
iteration.BP: INFO     SS: 0, SM: 0, results test algo (S,I,R): (200,0,0), infected test random: 0/0
iteration.BP: INFO     false+: 0 (+0), false-: 0 (+0)
iteration.BP: INFO     ...quarantining 0 guys -> got 0 infected, 0 free as birds (+0)


## Results
[see complete results in the output folder]

In [8]:
%matplotlib inline
to_plot = "I"
for s in ress.keys():
    plt.plot(ress[s][to_plot], label = s)
plt.semilogy()
plt.ylabel("Infected")
plt.xlabel("days")
plt.legend()
plt.show()